In [100]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from tqdm import tqdm

In [101]:
# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm


# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

En este laboratorio, partiremos de los datos con encodign y vamos a aplicar alguno de los métodos estudidados de cross validation, esperando así mejorar los resultados obtenidos en el laboratorio de regresion con el training y el test de una sola iteración.

In [102]:
df = pd.read_csv("datos/encoding.csv", index_col=0)

In [103]:
df.head()

,postalCode,price,powerCV_robust,kilometer_robust,seller,offerType,abtest,vehicleType,yearOfRegistration,gearbox,monthOfRegistration,fuelType,brand,notRepairedDamage
0,70435,480.0,-0.374913,0.0,5919.340419,5919.324221,5950.394844,5182.351533,2204.474969,4746.972781,4472.671667,5038.059231,5273.200085,3397.321601
1,66954,18300.0,1.128683,-1.0,5903.443396,5903.511051,5926.562881,12127.563144,7239.785288,4734.577120,6037.628327,8537.138905,8828.247228,2303.964735
2,90480,9800.0,0.746735,-1.0,5903.443396,5903.511051,5926.562881,13305.266694,7239.785288,10585.224181,6050.044119,8537.138905,12561.257062,3374.390103
3,91074,1500.0,-0.531319,0.0,5903.443396,5903.511051,5926.562881,2777.701282,7239.785288,4734.577120,6198.748774,5011.562951,5271.489409,7092.000540
4,60437,3600.0,-0.619461,-2.4,5916.765237,5916.724803,5958.128460,2778.258257,7246.674406,4756.336114,6710.554236,8521.338650,6409.241600,7117.079385


In [104]:
X = df.drop(columns="price")
y = df[["price"]]

In [105]:
modelo_lineal = LinearRegression()

Aplicamos el algoritmo Kfold, como creamos lotes y iteramos de manera que todos los datos pasen por test esperamos obtener resultados mejores que los que obtuvimos ayer para un número de lotes lo suficientemente alto. Empezamos, haciendo 5 lotes.

In [106]:
kf = KFold(n_splits=5, 
           shuffle=True, 
           random_state=42)

# RMSE
score_rmse = cross_val_score(modelo_lineal, 
                             X, 
                             y, 
                             cv=kf, 
                             scoring="neg_root_mean_squared_error")
rmse = abs(np.mean(score_rmse))

# MSE
score_mse = cross_val_score(modelo_lineal, 
                            X, 
                            y, 
                            cv=kf, 
                            scoring="neg_mean_squared_error")
mse = abs(np.mean(score_mse))

# MAE
score_mae = cross_val_score(modelo_lineal, 
                            X, 
                            y, 
                            cv=kf, 
                            scoring="neg_mean_absolute_error")
mae = abs(np.mean(score_mae))

# R^2
score_r2 = cross_val_score(modelo_lineal, 
                           X, 
                           y, 
                           cv=kf, 
                           scoring="r2")
r2 = np.mean(score_r2)

In [107]:
metricas_kfold = {'kf': {
        'r2_score': r2,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse}
}

In [108]:
pd.DataFrame(metricas_kfold).T

,r2_score,MAE,MSE,RMSE
kf,0.298227,3402.036369,1.022314e+08,10094.219753


Comparemos esto con los datos obtenidos en el train y en el test en el laboratorio anterior
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>r2_score</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>RMSE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.291759</td>
      <td>3388.629892</td>
      <td>1.033683e+08</td>
      <td>10167.020208</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.307367</td>
      <td>3398.433385</td>
      <td>9.953051e+07</td>
      <td>9976.497826</td>
    </tr>
  </tbody>
</table>
</div>

Vemos que los datos apenas mejoran los del train, y son peores que los del test. Vamos a subir el numero de lotes.

Vamos a aplicarlo ahora para 100 lotes

In [ ]:
kf = KFold(n_splits= 1000, 
           shuffle=True, 
           random_state=42)

# RMSE
score_rmse = cross_val_score(modelo_lineal, 
                             X, 
                             y, 
                             cv=kf, 
                             scoring="neg_root_mean_squared_error")
rmse = abs(np.mean(score_rmse))

# MSE
score_mse = cross_val_score(modelo_lineal, 
                            X, 
                            y, 
                            cv=kf, 
                            scoring="neg_mean_squared_error")
mse = abs(np.mean(score_mse))

# MAE
score_mae = cross_val_score(modelo_lineal, 
                            X, 
                            y, 
                            cv=kf, 
                            scoring="neg_mean_absolute_error")
mae = abs(np.mean(score_mae))

# R^2
score_r2 = cross_val_score(modelo_lineal, 
                           X, 
                           y, 
                           cv=kf, 
                           scoring="r2")
r2 = np.mean(score_r2)

In [111]:
metricas_kfold = {'kf': {
        'r2_score': r2,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse}
}

In [112]:
pd.DataFrame(metricas_kfold).T

,r2_score,MAE,MSE,RMSE
kf,0.399776,3401.763159,1.022401e+08,9089.537119


Comparemos esto con los datos obtenidos en el train y en el test en el laboratorio anterior
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>r2_score</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>RMSE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.291759</td>
      <td>3388.629892</td>
      <td>1.033683e+08</td>
      <td>10167.020208</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.307367</td>
      <td>3398.433385</td>
      <td>9.953051e+07</td>
      <td>9976.497826</td>
    </tr>
  </tbody>
</table>
</div>

In [113]:
df[["price"]].describe()

,price
count,371528.000000
mean,5923.847881
std,12053.036525
min,1.000000
25%,1250.000000
50%,3000.000000
75%,7300.000000
max,999999.000000


Aunque los datos son mejores que los del modelo del laboratorio anterior, siguen siendo malos, baste ver el RMSE obtenido con el precio medio de los coches. Teneos un RMSE mayor que lo que cuesta un coche de mierda.

En todo caso, el kfolds no es el modelo más adeucado para trabajar con los datos de nuestro laboratorio, dado que no se dan condiciones de independencia, normalidad y homoestaticidad y lo mismo pasaría con el LeaveOneOut que además tardará bastante en ejecutarse. Pasamos pues al decision tree.

## Decision tree